In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from scipy import stats

In [63]:
test=pd.read_csv('test.csv')
train=pd.read_csv('train.csv')

In [64]:
df=train.copy()#cкопируем, чтобы не трогать основно датасет
df.head()

,max_floor,state,marital_status,big_market_raion,total_revolving_bal,market_count_1500,leisure_count_3000,total_ct_chng_q4_q1,water_1line,railroad_station_walk_km,...,avg_open_to_buy,build_year,incineration_raion,full_sq,total_relationship_count,detention_facility_raion,build_count_mix,railroad_terminal_raion,__churn,__price_doc
0,NaN,NaN,Single,no,1906,1,0,0.694,no,5.419893,...,1996.9,NaN,no,43,5,no,0.0,no,0,5.850000
1,NaN,NaN,Married,no,12,0,6,0.693,no,3.411993,...,1338.4,NaN,no,34,3,no,0.0,no,0,6.000000
2,NaN,NaN,Married,no,2153,5,0,0.831,no,1.277658,...,2411.5,NaN,no,43,6,no,0.0,no,0,5.700000
3,NaN,NaN,Married,no,1806,2,0,0.638,no,4.291432,...,9255.9,NaN,no,89,3,no,2.0,no,0,13.100000
4,NaN,NaN,Married,no,753,2,40,0.902,no,0.853960,...,3318.2,NaN,no,77,4,no,1.0,yes,0,16.331452


In [65]:
df["timestamp"]

0        2011-08-20
1        2011-08-23
2        2011-08-27
3        2011-09-01
4        2011-09-05
            ...    
20478    2014-06-30
20479    2014-06-30
20480    2014-06-30
20481    2014-06-30
20482    2014-06-30
Name: timestamp, Length: 20483, dtype: object

In [66]:
df2=test.copy()#cкопируем, чтобы не трогать основной датасет
df2.head()

,max_floor,state,marital_status,big_market_raion,total_revolving_bal,market_count_1500,leisure_count_3000,total_ct_chng_q4_q1,water_1line,railroad_station_walk_km,...,months_on_book,dependent_count,avg_open_to_buy,build_year,incineration_raion,full_sq,total_relationship_count,detention_facility_raion,build_count_mix,railroad_terminal_raion
0,22.0,3.0,Married,no,616,0,1,0.296,yes,4.665296,...,40,2,23671.5,1979.0,no,54,3,no,0.0,no
1,16.0,3.0,Married,no,1556,1,3,0.671,no,3.516133,...,36,2,3320.3,1978.0,no,37,4,no,1.0,no
2,9.0,2.0,Married,no,1030,0,6,0.679,no,1.405934,...,36,3,9752.1,1972.0,no,44,2,no,0.0,no
3,17.0,3.0,Married,no,1508,0,0,0.732,no,4.434895,...,49,1,3113.3,2011.0,yes,38,4,no,0.0,no
4,48.0,1.0,Single,no,1059,0,1,0.840,yes,5.852092,...,36,2,203.3,2011.0,no,125,5,yes,0.0,no


In [67]:
#посмотрим на столбцы с существенной долей пропусков и удалим их
nas=train.isna().sum()/len(train)
list(nas[nas>0.1].index)

['max_floor',
 'state',
 '0_17_all',
 'build_count_wood',
 'life_sq',
 'cafe_sum_1000_min_price_avg',
 'total_trans_amt',
 'cafe_sum_1500_min_price_avg',
 'num_room',
 'build_year',
 'build_count_mix']

In [68]:
nas2=test.isna().sum()/len(test)
list(nas2[nas2>0.1].index)

['state',
 '0_17_all',
 'build_count_wood',
 'life_sq',
 'cafe_sum_1000_min_price_avg',
 'total_trans_amt',
 'cafe_sum_1500_min_price_avg',
 'build_year',
 'build_count_mix']

In [69]:
united_col=list(nas2[nas2>0.1].index)+list(nas[nas>0.1].index)

In [70]:
df=df.drop(united_col,axis=1)
df2=df2.drop(united_col,axis=1)

In [71]:
#остальные пропуски заменим на моды (это хорошая идея, 
#потому что мода естественным образом определена для категориальных признаков)
for i in df.columns:
    df[i].fillna(df[i].mode(), inplace=True)
for i in df2.columns:
    df2[i].fillna(df2[i].mode(), inplace=True)

In [72]:
#проверка
df.isna().sum()/len(df)

marital_status                           0.000000
big_market_raion                         0.000000
total_revolving_bal                      0.000000
market_count_1500                        0.000000
leisure_count_3000                       0.000000
total_ct_chng_q4_q1                      0.000000
water_1line                              0.000000
railroad_station_walk_km                 0.000488
culture_objects_top_25                   0.000000
contacts_count_12_mon                    0.000000
trc_count_2000                           0.000000
product_type                             0.000000
credit_limit                             0.000000
total_trans_ct                           0.000000
leisure_count_5000                       0.000000
cafe_count_1000_price_1000               0.000000
mkad_km                                  0.000000
school_education_centers_top_20_raion    0.000000
big_road1_1line                          0.000000
card_category                            0.000000


In [73]:
#проверка
df2.isna().sum()/len(df2)

marital_status                           0.000000
big_market_raion                         0.000000
total_revolving_bal                      0.000000
market_count_1500                        0.000000
leisure_count_3000                       0.000000
total_ct_chng_q4_q1                      0.000000
water_1line                              0.000000
railroad_station_walk_km                 0.001502
culture_objects_top_25                   0.000000
contacts_count_12_mon                    0.000000
trc_count_2000                           0.000000
product_type                             0.000000
credit_limit                             0.000000
total_trans_ct                           0.000000
leisure_count_5000                       0.000000
cafe_count_1000_price_1000               0.000000
mkad_km                                  0.000000
school_education_centers_top_20_raion    0.000000
big_road1_1line                          0.000000
card_category                            0.000000


In [74]:
#отберем количественные признаки и стандартизируем их
for i in df.select_dtypes(include=['float64',"int64"]).columns:
    df[i]=preprocessing.StandardScaler().fit_transform(df[[i]])
for i in df2.select_dtypes(include=['float64',"int64"]).columns:
    df2[i]=preprocessing.StandardScaler().fit_transform(df2[[i]])

In [61]:
non_cat_col=df.select_dtypes(include=['float64',"int64"]).columns

In [75]:
#поскольку они стандартизованы, можем легко убрать выбросы по z оценке
df[non_cat_col]=df[non_cat_col][df[non_cat_col]<3]
#df2[non_cat_col]=df2[non_cat_col][df2[non_cat_col]<3]

In [76]:
#отберем категориальные признаки и факторизируем их
for i in df.select_dtypes(include=["object"]).columns:
    df[i]=preprocessing.LabelEncoder().fit_transform(df[i])
for i in df2.select_dtypes(include=["object"]).columns:
    df2[i]=preprocessing.LabelEncoder().fit_transform(df2[i])

In [77]:
#видим, что средние нулевые, в бинарных признаках средние меньше 1, всё разумно выглядит
df.describe()

,marital_status,big_market_raion,total_revolving_bal,market_count_1500,leisure_count_3000,total_ct_chng_q4_q1,water_1line,railroad_station_walk_km,culture_objects_top_25,contacts_count_12_mon,...,months_on_book,dependent_count,avg_open_to_buy,incineration_raion,full_sq,total_relationship_count,detention_facility_raion,railroad_terminal_raion,__churn,__price_doc
count,20483.000000,20483.000000,2.048300e+04,20161.000000,19965.000000,20430.000000,20483.000000,19996.000000,20483.000000,20405.000000,...,2.048300e+04,2.048300e+04,19964.000000,20483.000000,20453.000000,2.048300e+04,20483.000000,20483.000000,2.048300e+04,20194.000000
mean,0.437387,0.092369,-1.617772e-16,-0.062876,-0.148169,-0.010615,0.080359,-0.095083,0.054484,-0.020295,...,-3.664385e-16,-2.456442e-16,-0.085287,0.089586,-0.013932,-2.092746e-16,0.100766,0.034663,-7.040211e-16,-0.074130
std,0.496076,0.289554,1.000024e+00,0.872366,0.340996,0.975552,0.271855,0.792529,0.226976,0.939658,...,1.000024e+00,1.000024e+00,0.859289,0.285595,0.470082,1.000024e+00,0.301027,0.182929,1.000024e+00,0.714937
min,0.000000,0.000000,-1.449619e+00,-0.684113,-0.291179,-3.844866,0.000000,-1.137883,0.000000,-4.215188,...,-3.188405e+00,-2.071508e+00,-0.921343,0.000000,-1.236302,-1.869980e+00,0.000000,0.000000,-4.386311e-01,-1.480938
25%,0.000000,0.000000,-7.145615e-01,-0.684113,-0.291179,-0.555958,0.000000,-0.634265,0.000000,-0.519139,...,-3.914603e-01,-1.173447e+00,-0.700234,0.000000,-0.362266,-4.195284e-01,0.000000,0.000000,-4.386311e-01,-0.506650
50%,0.000000,0.000000,7.272700e-02,-0.684113,-0.291179,0.063607,0.000000,-0.302181,0.000000,-0.519139,...,2.808142e-02,-2.753855e-01,-0.444358,0.000000,-0.132256,-4.195284e-01,0.000000,0.000000,-4.386311e-01,-0.178011
75%,1.000000,0.000000,7.695665e-01,0.192173,-0.129035,0.596317,0.000000,0.173248,0.000000,1.328885,...,4.476232e-01,6.226755e-01,0.231738,0.000000,0.212758,1.030923e+00,0.000000,0.000000,-4.386311e-01,0.208451
max,1.000000,1.000000,1.756862e+00,2.821028,2.951698,2.999304,1.000000,2.979518,1.000000,1.328885,...,2.825027e+00,2.418798e+00,2.999681,1.000000,2.995874,1.756149e+00,1.000000,1.000000,2.279820e+00,2.999456


In [79]:
df['timestamp']=train['timestamp']
df['__churn']=train['__churn']
df['__price_doc']=train['__price_doc']
df2['timestamp']=test['timestamp']

In [80]:
df2

,marital_status,big_market_raion,total_revolving_bal,market_count_1500,leisure_count_3000,total_ct_chng_q4_q1,water_1line,railroad_station_walk_km,culture_objects_top_25,contacts_count_12_mon,...,timestamp,education_level,months_on_book,dependent_count,avg_open_to_buy,incineration_raion,full_sq,total_relationship_count,detention_facility_raion,railroad_terminal_raion
0,0,0,-0.671994,-0.685651,-0.234314,-2.158285,1,0.030543,1,1.322431,...,2014-07-01,0,0.587100,-0.273784,2.047999,0,-0.050435,-0.414770,0,0
1,0,0,0.525274,0.243560,-0.098894,0.036025,0,-0.258405,0,-0.506486,...,2014-07-01,0,0.028841,-0.273784,-0.497901,0,-0.785576,0.306582,0,0
2,0,0,-0.144686,-0.685651,0.104236,0.082837,0,-0.788999,0,-2.335403,...,2014-07-01,0,0.028841,0.616370,0.306706,0,-0.482871,-1.136122,0,0
3,0,0,0.464137,-0.685651,-0.302024,0.392966,0,-0.027390,0,-0.506486,...,2014-07-01,0,1.843182,-1.163938,-0.523797,1,-0.742332,0.306582,0,0
4,1,0,-0.107749,-0.685651,-0.234314,1.024927,1,0.328954,0,-0.506486,...,2014-07-01,0,0.028841,-0.273784,-0.887833,0,3.019858,1.027933,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9983,1,0,-0.533161,1.172770,-0.302024,1.826581,0,-0.351181,0,-0.506486,...,2015-06-30,0,-0.529418,0.616370,-0.767488,1,-0.482871,1.027933,0,0
9984,0,0,0.117694,0.243560,2.948053,1.276541,0,-0.915652,1,-0.506486,...,2015-06-30,2,0.028841,1.506525,-0.532203,0,1.333359,0.306582,1,1
9985,0,0,1.239814,-0.685651,-0.302024,0.562659,0,-0.203206,0,-2.335403,...,2015-06-30,0,-0.529418,0.616370,-0.437554,0,-0.439627,-1.136122,0,0
9986,0,0,0.029809,-0.685651,-0.166604,-0.010787,0,0.591402,0,1.322431,...,2015-06-30,0,-2.204195,1.506525,-0.674002,0,0.382001,0.306582,0,0


In [81]:
#сохраним тестовый и трейновый датасеты
df.to_csv ('train_pr.csv', index= False )
df2.to_csv ('test_pr.csv', index= False )

In [14]:
#наконец, закончим на том, что посмотрим на сильно коррелирующие признаки
corr_mtrx=df.corr()
for i in corr_mtrx.columns:
    if len(list(corr_mtrx[(corr_mtrx[i]>0.8)|(corr_mtrx[i]<-0.8)].index))>1:
        print(i, " коррелирует с ",list(corr_mtrx[(corr_mtrx[i]>0.8)|(corr_mtrx[i]<-0.8)].index))

leisure_count_3000  коррелирует с  ['leisure_count_3000', 'leisure_count_5000']
credit_limit  коррелирует с  ['credit_limit', 'avg_open_to_buy']
leisure_count_5000  коррелирует с  ['leisure_count_3000', 'leisure_count_5000', 'office_sqm_5000']
office_sqm_5000  коррелирует с  ['leisure_count_5000', 'office_sqm_5000']
avg_open_to_buy  коррелирует с  ['credit_limit', 'avg_open_to_buy']


In [15]:
#ну и на как-нибудь коррелирующие
corr_mtrx=df.corr()
for i in corr_mtrx.columns:
    if len(list(corr_mtrx[(corr_mtrx[i]>0.5)|(corr_mtrx[i]<-0.5)].index))>1:
        print(i, " коррелирует с ",list(corr_mtrx[(corr_mtrx[i]>0.5)|(corr_mtrx[i]<-0.5)].index))

total_revolving_bal  коррелирует с  ['total_revolving_bal', 'avg_utilization_ratio']
leisure_count_3000  коррелирует с  ['leisure_count_3000', 'leisure_count_5000', 'cafe_count_1000_price_1000', 'sport_count_2000', 'office_sqm_5000']
railroad_station_walk_km  коррелирует с  ['railroad_station_walk_km', 'public_transport_station_min_walk', 'detention_facility_km']
trc_count_2000  коррелирует с  ['trc_count_2000', 'cafe_count_1000_price_1000', 'detention_facility_km', 'sport_count_2000']
product_type  коррелирует с  ['product_type', 'sport_count_2000']
credit_limit  коррелирует с  ['credit_limit', 'avg_open_to_buy']
leisure_count_5000  коррелирует с  ['leisure_count_3000', 'leisure_count_5000', 'cafe_count_1000_price_1000', 'sport_count_2000', 'office_sqm_5000']
cafe_count_1000_price_1000  коррелирует с  ['leisure_count_3000', 'trc_count_2000', 'leisure_count_5000', 'cafe_count_1000_price_1000', 'sport_count_2000', 'office_sqm_5000']
mkad_km  коррелирует с  ['mkad_km', 'metro_km_walk']
a